<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.sigmoid(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.sigmoid(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.sigmoid(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 90:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.09375
Current benign train loss: 2.321267604827881

Current batch: 100
Current benign train accuracy: 0.1640625
Current benign train loss: 4.7814154624938965

Current batch: 200
Current benign train accuracy: 0.2109375
Current benign train loss: 2.447993516921997

Current batch: 300
Current benign train accuracy: 0.1953125
Current benign train loss: 2.10707950592041

Total benign train accuarcy: 18.584
Total benign train loss: 1306.2264660596848

[ Test epoch: 0 ]

Test accuarcy: 20.85
Test average loss: 0.022363431572914122
Model Saved!

[ Train epoch: 1 ]

Current batch: 0
Current benign train accuracy: 0.2578125
Current benign train loss: 2.2044832706451416

Current batch: 100
Current benign train accuracy: 0.265625
Current benign train loss: 2.001647710800171

Current batch: 200
Current benign train accuracy: 0.265625
Current benign train loss: 2.2156102657318115

Current batch: 300
Current benign train accuracy


Current batch: 200
Current benign train accuracy: 0.734375
Current benign train loss: 0.7473959922790527

Current batch: 300
Current benign train accuracy: 0.671875
Current benign train loss: 0.7915477156639099

Total benign train accuarcy: 65.024
Total benign train loss: 385.75500589609146

[ Test epoch: 13 ]

Test accuarcy: 22.45
Test average loss: 0.08638196086883546
Model Saved!

[ Train epoch: 14 ]

Current batch: 0
Current benign train accuracy: 0.5625
Current benign train loss: 1.4814494848251343

Current batch: 100
Current benign train accuracy: 0.703125
Current benign train loss: 0.8290539979934692

Current batch: 200
Current benign train accuracy: 0.5703125
Current benign train loss: 1.1494446992874146

Current batch: 300
Current benign train accuracy: 0.6640625
Current benign train loss: 0.9118756055831909

Total benign train accuarcy: 66.59
Total benign train loss: 367.68920624256134

[ Test epoch: 14 ]

Test accuarcy: 33.66
Test average loss: 0.03950238354206085
Model Sav


Total benign train accuarcy: 74.07
Total benign train loss: 292.8332407772541

[ Test epoch: 26 ]

Test accuarcy: 20.27
Test average loss: 0.055389295625686646
Model Saved!

[ Train epoch: 27 ]

Current batch: 0
Current benign train accuracy: 0.6640625
Current benign train loss: 0.767824649810791

Current batch: 100
Current benign train accuracy: 0.6328125
Current benign train loss: 1.0254267454147339

Current batch: 200
Current benign train accuracy: 0.75
Current benign train loss: 0.7446898221969604

Current batch: 300
Current benign train accuracy: 0.7109375
Current benign train loss: 0.7735264301300049

Total benign train accuarcy: 74.686
Total benign train loss: 286.5379575788975

[ Test epoch: 27 ]

Test accuarcy: 17.63
Test average loss: 0.09202862010002136
Model Saved!

[ Train epoch: 28 ]

Current batch: 0
Current benign train accuracy: 0.71875
Current benign train loss: 0.8812803626060486

Current batch: 100
Current benign train accuracy: 0.765625
Current benign train loss: 


Current batch: 0
Current benign train accuracy: 0.71875
Current benign train loss: 0.7965640425682068

Current batch: 100
Current benign train accuracy: 0.7265625
Current benign train loss: 0.9159761667251587

Current batch: 200
Current benign train accuracy: 0.828125
Current benign train loss: 0.5575389862060547

Current batch: 300
Current benign train accuracy: 0.7734375
Current benign train loss: 0.6545895934104919

Total benign train accuarcy: 77.816
Total benign train loss: 254.14867594838142

[ Test epoch: 40 ]

Test accuarcy: 23.47
Test average loss: 0.05133640131950378
Model Saved!

[ Train epoch: 41 ]

Current batch: 0
Current benign train accuracy: 0.7734375
Current benign train loss: 0.7410234212875366

Current batch: 100
Current benign train accuracy: 0.7734375
Current benign train loss: 0.6343287229537964

Current batch: 200
Current benign train accuracy: 0.6953125
Current benign train loss: 0.6629354953765869

Current batch: 300
Current benign train accuracy: 0.7578125
C


Current batch: 200
Current benign train accuracy: 0.90625
Current benign train loss: 0.31740400195121765

Current batch: 300
Current benign train accuracy: 0.8359375
Current benign train loss: 0.4127732217311859

Total benign train accuarcy: 87.17
Total benign train loss: 146.36851555109024

[ Test epoch: 53 ]

Test accuarcy: 82.93
Test average loss: 0.005237222334742546
Model Saved!

[ Train epoch: 54 ]

Current batch: 0
Current benign train accuracy: 0.890625
Current benign train loss: 0.29637569189071655

Current batch: 100
Current benign train accuracy: 0.875
Current benign train loss: 0.46536532044410706

Current batch: 200
Current benign train accuracy: 0.859375
Current benign train loss: 0.32817134261131287

Current batch: 300
Current benign train accuracy: 0.859375
Current benign train loss: 0.5283024311065674

Total benign train accuarcy: 87.384
Total benign train loss: 144.0629026144743

[ Test epoch: 54 ]

Test accuarcy: 81.98
Test average loss: 0.0056432500258088115
Model 


Total benign train accuarcy: 88.522
Total benign train loss: 130.78464518487453

[ Test epoch: 66 ]

Test accuarcy: 77.46
Test average loss: 0.007051112014055252
Model Saved!

[ Train epoch: 67 ]

Current batch: 0
Current benign train accuracy: 0.9375
Current benign train loss: 0.22583089768886566

Current batch: 100
Current benign train accuracy: 0.890625
Current benign train loss: 0.38849759101867676

Current batch: 200
Current benign train accuracy: 0.90625
Current benign train loss: 0.3062797784805298

Current batch: 300
Current benign train accuracy: 0.796875
Current benign train loss: 0.5854942798614502

Total benign train accuarcy: 88.444
Total benign train loss: 130.6991719007492

[ Test epoch: 67 ]

Test accuarcy: 67.37
Test average loss: 0.01200372632741928
Model Saved!

[ Train epoch: 68 ]

Current batch: 0
Current benign train accuracy: 0.8828125
Current benign train loss: 0.4128398299217224

Current batch: 100
Current benign train accuracy: 0.8828125
Current benign train 


Current batch: 0
Current benign train accuracy: 0.90625
Current benign train loss: 0.2570751905441284

Current batch: 100
Current benign train accuracy: 0.90625
Current benign train loss: 0.26282188296318054

Current batch: 200
Current benign train accuracy: 0.890625
Current benign train loss: 0.29674315452575684

Current batch: 300
Current benign train accuracy: 0.890625
Current benign train loss: 0.2615223824977875

Total benign train accuarcy: 88.862
Total benign train loss: 125.31896574795246

[ Test epoch: 80 ]

Test accuarcy: 64.51
Test average loss: 0.013718287807703018
Model Saved!

[ Train epoch: 81 ]

Current batch: 0
Current benign train accuracy: 0.8515625
Current benign train loss: 0.4750809669494629

Current batch: 100
Current benign train accuracy: 0.90625
Current benign train loss: 0.32767578959465027

Current batch: 200
Current benign train accuracy: 0.90625
Current benign train loss: 0.2496236264705658

Current batch: 300
Current benign train accuracy: 0.859375
Curre


Current batch: 200
Current benign train accuracy: 0.90625
Current benign train loss: 0.2714758515357971

Current batch: 300
Current benign train accuracy: 0.9375
Current benign train loss: 0.2011636346578598

Total benign train accuarcy: 91.768
Total benign train loss: 95.52677192538977

[ Test epoch: 93 ]

Test accuarcy: 88.5
Test average loss: 0.0035197603166103363
Model Saved!

[ Train epoch: 94 ]

Current batch: 0
Current benign train accuracy: 0.90625
Current benign train loss: 0.23810593783855438

Current batch: 100
Current benign train accuracy: 0.8828125
Current benign train loss: 0.2826680541038513

Current batch: 200
Current benign train accuracy: 0.890625
Current benign train loss: 0.26310861110687256

Current batch: 300
Current benign train accuracy: 0.8984375
Current benign train loss: 0.267311692237854

Total benign train accuarcy: 91.798
Total benign train loss: 94.30567268282175

[ Test epoch: 94 ]

Test accuarcy: 87.59
Test average loss: 0.003890788158774376
Model Sav